In [6]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model
%cd first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 293, done.
remote: Total 293 (delta 0), reused 0 (delta 0), pack-reused 293
Receiving objects: 100% (293/293), 72.14 MiB | 37.65 MiB/s, done.
Resolving deltas: 100% (151/151), done.
/content/first-order-model/first-order-model


In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [8]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))
 
    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])
 
    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

In [10]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/gdrive/My Drive/First Order Model/vox-cpk.pth.tar')

In [11]:
#@title Set path for the video and image in your drive
face_path = "First Order Model/swapnil.jpeg" #@param {type:"string"}
driver_path = "First Order Model/driver.mp4" #@param {type:"string"}

In [12]:
from demo import make_animation
from skimage import img_as_ubyte
 
source_image = imageio.imread('/content/gdrive/My Drive/'+face_path)
reader = imageio.get_reader('/content/gdrive/My Drive/'+driver_path)
 
fps = reader.get_meta_data()['fps']
driving_video = []
try:
    for im in reader:
        driving_video.append(im)
except RuntimeError:
    pass
reader.close()
 
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]
 
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True,
                             adapt_movement_scale=True)

imageio.mimsave('../output.mp4', [img_as_ubyte(frame) for frame in predictions])
 
HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 852/852 [00:35<00:00, 24.32it/s]


## 8. Download the resulted video

In [1]:
from google.colab import files
files.download("/content/output.mp4")

FileNotFoundError: ignored